In [46]:
import pandas as pd 
import numpy as np 

In [4]:
as_lst = pd.read_csv("../res/asset_list.csv",encoding="gbk")['Symbol'].to_list()

In [8]:
print(as_lst)

['XAU', 'IRX', 'FVX', 'TNX', 'TYX', 'IXIC', 'GSPC', 'DJI', 'VIX', 'USDX', 'ZSF', 'ZWF', 'KCU', 'LBN', 'LCC', 'LEQ', 'LRCU', 'LSU', 'NGQ', 'RBQ', 'ZCU']


In [17]:
sub_set = ['XAU', 'IRX', 'FVX', 'TNX', 'TYX', 'IXIC', 'GSPC', 'DJI', 'VIX', 'USDX', 'KCU', 'LBN', 'ZCU']
as_lst = sub_set

In [42]:
all_asset = {}
for ast in as_lst:
    this_asset = pd.read_csv(f"../res/origin_series/{ast}.csv")
    this_asset['Date'] = pd.to_datetime(this_asset['Date'])
    all_asset[ast] = this_asset[['Date','Close']].set_index('Date')
    
close_df = pd.concat(all_asset,axis=1,join='outer')
close_df.columns=as_lst
close_df

,XAU,IRX,FVX,TNX,TYX,IXIC,GSPC,DJI,VIX,USDX,KCU,LBN,ZCU
Date,,,,,,,,,,,,,
1978-12-29,226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-01,226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-02,226.8,9.410,9.330,9.180,8.990,117.839996,96.730003,NaN,NaN,86.519997,NaN,NaN,NaN
1979-01-03,218.6,9.310,9.330,9.160,8.960,119.110001,97.800003,NaN,NaN,87.410004,NaN,NaN,NaN
1979-01-04,223.2,9.310,9.270,9.110,8.940,120.660004,98.580002,NaN,NaN,87.540001,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-06,"1,940.80",0.083,0.363,0.820,1.599,11895.230470,3509.439941,28323.400391,24.860001,92.230003,106.95,555.2,406.75
2020-11-09,NaN,0.088,0.447,0.958,1.751,11713.780270,3550.500000,29157.970703,25.750000,92.730003,106.85,585.0,407.50
2020-11-10,NaN,0.090,0.460,0.972,1.760,11553.860350,3545.530029,29420.919922,24.799999,92.750000,107.30,580.0,423.00


In [51]:
count_na = close_df.isnull().sum(axis=1)
# valid_idx = count_na[count_na==0].index[0]
close_df = close_df.loc[count_na[count_na==0].index[0]:pd.to_datetime('2020-11-06 00:00:00')]
# close_df = close_df.dropna(subset=['XAU'])
close_df['XAU'] = close_df['XAU'].apply(lambda x: x.replace(',','') if type(x)==str else x).astype(float)
close_df.fillna(method='ffill',inplace=True)
close_df = close_df.replace(0,1e-6)
close_df[close_df<0] = 0
# close_df = close_df.fillna(1e-6)
assert close_df.isnull().sum().sum() == 0
close_df.to_csv('../res/close_df.csv')
# logr_df = np.log(close_df).diff()
# logr_df['label'] = 0
# logr_df['label'][logr_df.XAU > 0] = 1

In [40]:
pd.to_datetime('2020-11-06 00:00:00')

Timestamp('2020-11-06 00:00:00')

In [11]:
count_na = close_df.isnull().sum(axis=1)


close_df = close_df.dropna(subset=['XAU'])
close_df['XAU'] = close_df['XAU'].apply(lambda x: x.replace(',','')).astype(float)
close_df = close_df.fillna(1e-6)

logr_df = np.log(close_df).diff()
logr_df['label'] = 0
logr_df['label'][logr_df.XAU > 0] = 1

In [12]:
logr_df[valid_idx:]

,XAU,IRX,FVX,TNX,TYX,IXIC,GSPC,DJI,VIX,USDX,KCU,LBN,NGQ,ZCU,label
Date,,,,,,,,,,,,,,,
1990-04-04,-0.001868,-0.008992,-0.009302,-0.009313,-0.010508,-0.005269,-0.007448,-0.006356,0.074911,-0.000319,0.031657,0.010706,14.297937,0.006545,0
1990-04-05,0.010894,0.001289,0.001168,0.002336,0.000000,-0.004558,-0.001056,0.000662,-0.037932,-0.001278,0.018359,0.005058,-0.003091,0.007428,1
1990-04-06,-0.001852,0.000000,-0.002336,-0.001168,0.000000,-0.005831,-0.001910,-0.001489,0.025601,-0.001493,-0.011226,-0.001515,0.006173,-0.003707,0
1990-04-09,-0.002651,0.007702,0.003503,0.003499,0.003515,-0.001626,0.003786,0.001820,0.024513,-0.000427,0.015146,0.008052,0.007357,0.000928,0
1990-04-10,-0.003990,-0.005128,0.000000,0.001163,0.002336,0.003944,0.002048,0.003305,-0.027735,-0.007285,0.006567,-0.020765,-0.010439,0.011987,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-02,0.004242,0.000000,-0.010554,-0.012873,-0.009804,0.004209,0.012243,0.015852,-0.023687,0.000106,-0.015444,0.022742,0.000000,-0.002513,1
2020-11-03,0.009689,0.080969,0.041564,0.038133,0.018909,0.018353,0.017643,0.020402,-0.043485,-0.005331,0.006303,0.056369,0.000000,0.008766,1
2020-11-04,-0.004254,-0.080969,-0.196157,-0.138402,-0.068130,0.037823,0.021808,0.013289,-0.184180,-0.001498,-0.004359,0.023218,0.000000,0.010543,0


In [ ]:
final_df = logr_df[valid_idx:]

In [91]:
logr_df[valid_idx:].drop("XAU",axis=1).to_csv('../res/logr_label.csv')